# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/surohittandon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/surohittandon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/surohittandon/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# Import libraries
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/surohittandon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
# import libraries 
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

In [4]:
# connect to the database
engine = create_engine('sqlite:///InsertDatabaseName.db')
# load data from database
df = pd.read_sql_table('InsertTableName', engine)

In [5]:
# Checking df was loading in correctly 
df.head()


,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df[list(df.columns[5:])].shape

(26207, 36)

In [7]:
df.message.shape

(26207,)

In [9]:
# Splitting the data into predictors and targets
X = df.message
Y = df[list(df.columns[5:])]
# Mapping the '2' values in 'related' to '1' - because they are considered as a response 
Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # Converting all text to lower case
    text=text.lower()
    
    # Removing punctuation from all text
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    words = word_tokenize(text)
    
    # Removing stop words
    new_words = []
    for w in words:
        if w not in stopwords.words('english'):
            new_words.append(w)
    
    # Lemmatizing our new words
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for word in new_words:
        clean_tok = lemmatizer.lemmatize(word).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [74]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [64]:
y_actual_test = y_test.values
y_pred_test = pipeline.predict(X_test)

In [65]:
y_pred_pd = pd.DataFrame(y_pred_test, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

         0.0       0.30      0.15      0.20      1507
         1.0       0.78      0.90      0.84      5045

   micro avg       0.73      0.73      0.73      6552
   macro avg       0.54      0.52      0.52      6552
weighted avg       0.67      0.73      0.69      6552

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      5413
         1.0       0.44      0.09      0.15      1139

   micro avg       0.82      0.82      0.82      6552
   macro avg       0.64      0.53      0.52      6552
weighted avg       0.77      0.82      0.77      6552

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0     

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6515
         1.0       0.00      0.00      0.00        37

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.99      0.99      0.99      6552

------------------------------------------------------

FEATURE: hospitals

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6490
         1.0       0.00      0.00      0.00        62

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.98      0.99      0.99      6552

------------------------------------------------------

FEATURE: shops

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0       0.00      0.00      0.00        26

   micro avg       1.00      1.00 

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:

overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))


Average overall accuracy 92.27% 



### 6. Improve your model
Use grid search to find better parameters. 

In [81]:
parameters = {'clf__max_depth': [None],
              'clf__min_samples_leaf': [2],
              'clf__min_samples_split': [2,3],
              'clf__n_estimators': [15,20,25]}


cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=5, n_jobs=-1)


In [82]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed:  5.1min remaining:  8.0min
[Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed: 10.1min remaining:  6.4min
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed: 10.1min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 12.4min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__max_depth': [None], 'clf__min_samples_leaf': [2], 'clf__min_samples_split': [2, 3], 'clf__n_estimators': [15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

In [83]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 2,
 'clf__min_samples_split': 3,
 'clf__n_estimators': 15}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [84]:
y_pred_grid_test = cv.predict(X_test)

In [85]:
y_pred_grid_pd = pd.DataFrame(y_pred_grid_test, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_grid_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

         0.0       0.49      0.02      0.04      1507
         1.0       0.77      0.99      0.87      5045

   micro avg       0.77      0.77      0.77      6552
   macro avg       0.63      0.51      0.46      6552
weighted avg       0.71      0.77      0.68      6552

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

         0.0       0.83      1.00      0.90      5413
         1.0       0.32      0.01      0.01      1139

   micro avg       0.82      0.82      0.82      6552
   macro avg       0.57      0.50      0.46      6552
weighted avg       0.74      0.82      0.75      6552

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0     

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6490
         1.0       0.00      0.00      0.00        62

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.98      0.99      0.99      6552

------------------------------------------------------

FEATURE: shops

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.99      1.00      0.99      6552

------------------------------------------------------

FEATURE: aid_centers

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6476
         1.0       0.00      0.00      0.00        76

   micro avg       0.99      0.9

In [95]:
overall_accuracy = (y_pred_grid_test == y_test).mean().mean()

print('\n\nAverage overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))





Average overall accuracy 92.67% 



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [90]:
pipeline2 = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf_2', MultiOutputClassifier(AdaBoostClassifier()))
                     ])

pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

In [91]:
y_pred_ada_test = pipeline2.predict(X_test)

In [92]:
overall_accuracy = (y_pred_ada_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))



Average overall accuracy 92.60% 



In [93]:
y_pred_grid_pd = pd.DataFrame(y_pred_ada_test, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_grid_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

         0.0       0.37      0.04      0.07      1507
         1.0       0.77      0.98      0.86      5045

   micro avg       0.76      0.76      0.76      6552
   macro avg       0.57      0.51      0.47      6552
weighted avg       0.68      0.76      0.68      6552

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      5413
         1.0       0.47      0.10      0.17      1139

   micro avg       0.82      0.82      0.82      6552
   macro avg       0.66      0.54      0.54      6552
weighted avg       0.77      0.82      0.77      6552

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0     

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6490
         1.0       0.00      0.00      0.00        62

   micro avg       0.99      0.99      0.99      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.98      0.99      0.99      6552

------------------------------------------------------

FEATURE: shops

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6526
         1.0       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.99      1.00      0.99      6552

------------------------------------------------------

FEATURE: aid_centers

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6476
         1.0       0.00      0.00      0.00        76

   micro avg       0.99      0.9

### 9. Export your model as a pickle file

In [98]:
filename = 'dis_classifier.sav'
pickle.dump(cv, open(filename, 'wb'))

In [99]:
cv

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__max_depth': [None], 'clf__min_samples_leaf': [2], 'clf__min_samples_split': [2, 3], 'clf__n_estimators': [15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

In [100]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [101]:
loaded_model

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__max_depth': [None], 'clf__min_samples_leaf': [2], 'clf__min_samples_split': [2, 3], 'clf__n_estimators': [15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.